In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [3]:
# Load data
data = pd.read_csv('TSLA_stock_data_2023.csv')
# data_day = pd.read.csv('TSLA_stock_data_day_2023.csv')

In [16]:


# Initialize empty lists for states and predict_price
states = []
predict_prices = []

# Set your rolling window size (3 days)
rolling_window_size = 3

# Start your window from the beginning of the data
current_start_index = 0

# Calculate the end index for the first window
current_end_index = rolling_window_size * 7  # assuming hourly data
while current_end_index < len(data)-7:
    # Get the current window of data
    current_window = data.iloc[current_start_index:current_end_index]
    
    # Flatten the data from the current window and append it to states
    window_features = current_window[['Open', 'High', 'Low', 'Close', 'Volume']].values.flatten()
    states.append(window_features)
    
    # We assume 'Close' price is what we want to predict and use the last 'Close' price of the current window
    predict_prices.append(data.iloc[current_end_index+6]['Close'])
    
    # Move the window one day forward
    current_start_index += 1  # assuming hourly data
    current_end_index += 1
# Convert the lists to NumPy arrays
states = np.array(states)
predict_prices = np.array(predict_prices)

# Convert arrays to tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
states_tensor = torch.tensor(states, dtype=torch.float32).to(device)
predict_prices_tensor = torch.tensor(predict_prices, dtype=torch.float32).view(-1, 1).to(device)

# Now you can check the first state and price
print(states_tensor[0])  # First state
print(predict_prices_tensor[0])  # First price to predict

tensor([1.1847e+02, 1.1880e+02, 1.1035e+02, 1.1106e+02, 6.5616e+07, 1.1105e+02,
        1.1224e+02, 1.0805e+02, 1.0847e+02, 3.3929e+07, 1.0847e+02, 1.0850e+02,
        1.0560e+02, 1.0615e+02, 3.3451e+07, 1.0615e+02, 1.0648e+02, 1.0464e+02,
        1.0567e+02, 2.4088e+07, 1.0567e+02, 1.0745e+02, 1.0534e+02, 1.0732e+02,
        2.3360e+07, 1.0732e+02, 1.0940e+02, 1.0677e+02, 1.0808e+02, 2.9563e+07,
        1.0809e+02, 1.0875e+02, 1.0752e+02, 1.0806e+02, 1.5564e+07, 1.0911e+02,
        1.1274e+02, 1.0752e+02, 1.1072e+02, 6.0188e+07, 1.1069e+02, 1.1244e+02,
        1.0993e+02, 1.1230e+02, 2.6007e+07, 1.1232e+02, 1.1459e+02, 1.1225e+02,
        1.1275e+02, 2.5649e+07, 1.1275e+02, 1.1294e+02, 1.1143e+02, 1.1276e+02,
        1.4665e+07, 1.1276e+02, 1.1430e+02, 1.1184e+02, 1.1203e+02, 2.3124e+07,
        1.1202e+02, 1.1394e+02, 1.1190e+02, 1.1301e+02, 1.6542e+07, 1.1301e+02,
        1.1372e+02, 1.1225e+02, 1.1364e+02, 1.0188e+07, 1.1051e+02, 1.1133e+02,
        1.0744e+02, 1.0842e+02, 4.9873e+

In [152]:
assert state.shape[0] == predict_price.shape[0], "The number of samples should be the same."
# Now, use train_test_split
state_train, state_test, predict_price_train, predict_price_test = train_test_split(
    state, predict_price, test_size=0.2, random_state=42
)

# # divide the data into training part and test part
# state_train, state_test, predict_price_train,predict_price_test= train_test_split(state, predict_price,test_size=0.2, random_state=42)
# train_loader = DataLoader(TensorDataset(state_train, predict_price_train), batch_size=5, shuffle=True)
# test_loader = DataLoader(TensorDataset(state_test, predict_price_test), batch_size=1)
state_train

array([[2.54490005e+02, 2.56980011e+02, 2.52910004e+02, 2.56170013e+02,
        2.35974050e+07, 2.56160004e+02, 2.57479004e+02, 2.55389999e+02,
        2.56394989e+02, 1.63013580e+07, 2.56409698e+02, 2.57519012e+02,
        2.56100006e+02, 2.57445007e+02, 1.09132680e+07, 2.57440186e+02,
        2.57820007e+02, 2.56589996e+02, 2.57720001e+02, 1.00004970e+07,
        2.57730011e+02, 2.57970001e+02, 2.57210205e+02, 2.57502991e+02,
        7.48659400e+06, 2.57510010e+02, 2.57630005e+02, 2.56179993e+02,
        2.56855011e+02, 9.02844900e+06, 2.56869995e+02, 2.57459991e+02,
        2.56339996e+02, 2.56709991e+02, 6.31647400e+06, 2.58350006e+02,
        2.63339996e+02, 2.57519989e+02, 2.62839996e+02, 3.54315110e+07,
        2.62829987e+02, 2.62950012e+02, 2.60690002e+02, 2.62250000e+02,
        1.76088960e+07, 2.62244995e+02, 2.62959991e+02, 2.61920013e+02,
        2.62464996e+02, 1.03576300e+07, 2.62480011e+02, 2.62920013e+02,
        2.61829987e+02, 2.62842896e+02, 9.86631300e+06, 2.628500

In [20]:
# construct NN
class NN(nn.Module):
    def __init__(self, n_observations):
        super(NN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
# set some initail parameters 
LR = 1e-3
Weight_decay = 0.01
n_observations = len(state[0])


In [ ]:
# value_net is your prediction function, take state as input and output is the prediction price
value_net = NN(n_observations).to(device)

In [ ]:
# 'optimize'  is an easy package to do Gredient decent, 'Adam' is a method to let learing rate decay as step go.
optimizer = optim.Adam(value_net.parameters(), lr=LR, weight_decay = Weight_decay)


In [ ]:
# Optimazation function, it do one step of gredient decent, 
def optimize_model():
    for state, target in train_loader:
        current_value=value_net(state)                
        criterion = nn.SmoothL1Loss()
        loss=criterion(current_value,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    for state, target in test_loader:
        with torch.no_grad():
            test_valur=value_net(state)                
            l_test=criterion(current_value,target)
    return loss.item(),l_test.item()

In [ ]:
# training
epochs=1000
for epoch in range(epochs):
    l_train,l_test=optimize_model()
    if epoch % 100 ==0:
        print(f'Epoch [{epoch+1}/{epochs}], L_train: {l_train},L_test: {l_test}')
